In [ ]:

#!/usr/bin/env python
import copy
import torch
import argparse
import os
import time
import warnings
import numpy as np
import torchvision
import logging

from flcore.servers.serveravg import FedAvg
from flcore.servers.serverpFedMe import pFedMe
from flcore.servers.serverperavg import PerAvg
from flcore.servers.serverprox import FedProx
from flcore.servers.serverfomo import FedFomo
from flcore.servers.serveramp import FedAMP
from flcore.servers.servermtl import FedMTL
from flcore.servers.serverlocal import Local
from flcore.servers.serverper import FedPer
from flcore.servers.serverapfl import APFL
from flcore.servers.serverditto import Ditto
from flcore.servers.serverrep import FedRep
from flcore.servers.serverphp import FedPHP
from flcore.servers.serverbn import FedBN
from flcore.servers.serverrod import FedROD
from flcore.servers.serverproto import FedProto
from flcore.servers.serverdyn import FedDyn
from flcore.servers.servermoon import MOON
from flcore.servers.serverbabu import FedBABU
from flcore.servers.serverapple import APPLE
from flcore.servers.serverfedtrans import FedTrans 

from flcore.trainmodel.models import *

from flcore.trainmodel.bilstm import BiLSTM_TextClassification
# from flcore.trainmodel.resnet import resnet18 as resnet
from flcore.trainmodel.alexnet import alexnet
from flcore.trainmodel.mobilenet_v2 import mobilenet_v2
from utils.result_utils import average_data
from utils.mem_utils import MemReporter

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

warnings.simplefilter("ignore")
torch.manual_seed(0)

# hyper-params for Text tasks
vocab_size = 98635
max_len=200
hidden_dim=32

def run(args):

    time_list = []
    reporter = MemReporter()
    model_str = args.model

    for i in range(args.prev, args.times):
        print(f"\n============= Running time: {i}th =============")
        print("Creating server and clients ...")
        start = time.time()

        # Generate args.model
        if model_str == "mlr":
            if args.dataset == "mnist" or args.dataset == "fmnist":
                args.model = Mclr_Logistic(1*28*28, num_classes=args.num_classes).to(args.device)
            elif args.dataset == "Cifar10" or args.dataset == "Cifar100":
                args.model = Mclr_Logistic(3*32*32, num_classes=args.num_classes).to(args.device)
            else:
                args.model = Mclr_Logistic(60, num_classes=args.num_classes).to(args.device)

        elif model_str == "cnn":
            if args.dataset[:5] == "mnist" or args.dataset == "fmnist":
                args.model = FedAvgCNN(in_features=1, num_classes=args.num_classes, dim=1024).to(args.device)
            elif args.dataset == "omniglot":
                args.model = FedAvgCNN(in_features=1, num_classes=args.num_classes, dim=33856).to(args.device)
            elif args.dataset[:5] == "Cifar":
                args.model = FedAvgCNN(in_features=3, num_classes=args.num_classes, dim=1600).to(args.device)
                # args.model = CifarNet(num_classes=args.num_classes).to(args.device)
            elif args.dataset == "Digit5":
                args.model = Digit5CNN().to(args.device)
            else:
                args.model = FedAvgCNN(in_features=3, num_classes=args.num_classes, dim=10816).to(args.device)

        elif model_str == "dnn": # non-convex
            if args.dataset == "mnist" or args.dataset == "fmnist":
                args.model = DNN(1*28*28, 100, num_classes=args.num_classes).to(args.device)
            elif args.dataset == "Cifar10" or args.dataset == "Cifar100":
                args.model = DNN(3*32*32, 100, num_classes=args.num_classes).to(args.device)
            else:
                args.model = DNN(60, 20, num_classes=args.num_classes).to(args.device)
        
        elif model_str == "resnet":
            args.model = torchvision.models.resnet18(pretrained=False, num_classes=args.num_classes).to(args.device)
            
            # args.model = torchvision.models.resnet18(pretrained=True).to(args.device)
            # feature_dim = list(args.model.fc.parameters())[0].shape[1]
            # args.model.fc = nn.Linear(feature_dim, args.num_classes).to(args.device)
            
            # args.model = resnet18(num_classes=args.num_classes, has_bn=True, bn_block_num=4).to(args.device)

        elif model_str == "alexnet":
            args.model = alexnet(pretrained=False, num_classes=args.num_classes).to(args.device)
            
            # args.model = alexnet(pretrained=True).to(args.device)
            # feature_dim = list(args.model.fc.parameters())[0].shape[1]
            # args.model.fc = nn.Linear(feature_dim, args.num_classes).to(args.device)
            
        elif model_str == "googlenet":
            args.model = torchvision.models.googlenet(pretrained=False, aux_logits=False, num_classes=args.num_classes).to(args.device)
            
            # args.model = torchvision.models.googlenet(pretrained=True, aux_logits=False).to(args.device)
            # feature_dim = list(args.model.fc.parameters())[0].shape[1]
            # args.model.fc = nn.Linear(feature_dim, args.num_classes).to(args.device)

        elif model_str == "mobilenet_v2":
            args.model = mobilenet_v2(pretrained=False, num_classes=args.num_classes).to(args.device)
            
            # args.model = mobilenet_v2(pretrained=True).to(args.device)
            # feature_dim = list(args.model.fc.parameters())[0].shape[1]
            # args.model.fc = nn.Linear(feature_dim, args.num_classes).to(args.device)
            
        elif model_str == "lstm":
            args.model = LSTMNet(hidden_dim=hidden_dim, vocab_size=vocab_size, num_classes=args.num_classes).to(args.device)

        elif model_str == "bilstm":
            args.model = BiLSTM_TextClassification(input_size=vocab_size, hidden_size=hidden_dim, output_size=args.num_classes, 
                        num_layers=1, embedding_dropout=0, lstm_dropout=0, attention_dropout=0, 
                        embedding_length=hidden_dim).to(args.device)

        elif model_str == "fastText":
            args.model = fastText(hidden_dim=hidden_dim, vocab_size=vocab_size, num_classes=args.num_classes).to(args.device)

        elif model_str == "TextCNN":
            args.model = TextCNN(hidden_dim=hidden_dim, max_len=max_len, vocab_size=vocab_size, 
                            num_classes=args.num_classes).to(args.device)

        elif model_str == "Transformer":
            args.model = TransformerModel(ntoken=vocab_size, d_model=hidden_dim, nhead=2, d_hid=hidden_dim, nlayers=2, 
                            num_classes=args.num_classes).to(args.device)
        
        elif model_str == "AmazonMLP":
            args.model = AmazonMLP().to(args.device)

        else:
            raise NotImplementedError

        print(args.model)

        # select algorithm
        if args.algorithm == "FedAvg":
            server = FedAvg(args, i)

        elif args.algorithm == "Local":
            server = Local(args, i)

        elif args.algorithm == "FedMTL":
            server = FedMTL(args, i)

        elif args.algorithm == "PerAvg":
            server = PerAvg(args, i)

        elif args.algorithm == "pFedMe":
            server = pFedMe(args, i)

        elif args.algorithm == "FedProx":
            server = FedProx(args, i)

        elif args.algorithm == "FedFomo":
            server = FedFomo(args, i)

        elif args.algorithm == "FedAMP":
            server = FedAMP(args, i)

        elif args.algorithm == "APFL":
            server = APFL(args, i)

        elif args.algorithm == "FedPer":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedPer(args, i)

        elif args.algorithm == "Ditto":
            server = Ditto(args, i)

        elif args.algorithm == "FedRep":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedRep(args, i)

        elif args.algorithm == "FedPHP":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedPHP(args, i)

        elif args.algorithm == "FedBN":
            server = FedBN(args, i)

        elif args.algorithm == "FedROD":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedROD(args, i)

        elif args.algorithm == "FedProto":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedProto(args, i)

        elif args.algorithm == "FedDyn":
            server = FedDyn(args, i)

        elif args.algorithm == "MOON":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = MOON(args, i)

        elif args.algorithm == "FedBABU":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedBABU(args, i)

        elif args.algorithm == "APPLE":
            server = APPLE(args, i)
            
        elif args.algorithm == "FedTrans":
            args.head = copy.deepcopy(args.model.fc)
            args.model.fc = nn.Identity()
            args.model = LocalModel(args.model, args.head)
            server = FedTrans(args, i)
            
        else:
            raise NotImplementedError
    
    return server

if __name__ == "__main__":
    total_start = time.time()

    parser = argparse.ArgumentParser()
    # general
    parser.add_argument('-go', "--goal", type=str, default="test", 
                        help="The goal for this experiment")
    parser.add_argument('-dev', "--device", type=str, default="cuda",
                        choices=["cpu", "cuda"])
    parser.add_argument('-did', "--device_id", type=str, default="0")
    parser.add_argument('-data', "--dataset", type=str, default="mnist")
    parser.add_argument('-nb', "--num_classes", type=int, default=10)
    parser.add_argument('-m', "--model", type=str, default="cnn")
    parser.add_argument('-p', "--head", type=str, default="cnn")
    parser.add_argument('-lbs', "--batch_size", type=int, default=10)
    parser.add_argument('-lr', "--local_learning_rate", type=float, default=0.005,
                        help="Local learning rate")
    parser.add_argument('-gr', "--global_rounds", type=int, default=1000)
    parser.add_argument('-ls', "--local_steps", type=int, default=1)
    parser.add_argument('-algo', "--algorithm", type=str, default="FedAvg")
    parser.add_argument('-jr', "--join_ratio", type=float, default=1.0,
                        help="Ratio of clients per round")
    parser.add_argument('-rjr', "--random_join_ratio", type=bool, default=False,
                        help="Random ratio of clients per round")
    parser.add_argument('-nc', "--num_clients", type=int, default=2,
                        help="Total number of clients")
    parser.add_argument('-pv', "--prev", type=int, default=0,
                        help="Previous Running times")
    parser.add_argument('-t', "--times", type=int, default=1,
                        help="Running times")
    parser.add_argument('-eg', "--eval_gap", type=int, default=1,
                        help="Rounds gap for evaluation")
    parser.add_argument('-dp', "--privacy", type=bool, default=False,
                        help="differential privacy")
    parser.add_argument('-dps', "--dp_sigma", type=float, default=0.0)
    parser.add_argument('-sfn', "--save_folder_name", type=str, default='models')
    # practical
    parser.add_argument('-cdr', "--client_drop_rate", type=float, default=0.0,
                        help="Rate for clients that train but drop out")
    parser.add_argument('-tsr', "--train_slow_rate", type=float, default=0.0,
                        help="The rate for slow clients when training locally")
    parser.add_argument('-ssr', "--send_slow_rate", type=float, default=0.0,
                        help="The rate for slow clients when sending global model")
    parser.add_argument('-ts', "--time_select", type=bool, default=False,
                        help="Whether to group and select clients at each round according to time cost")
    parser.add_argument('-tth', "--time_threthold", type=float, default=10000,
                        help="The threthold for droping slow clients")
    # pFedMe / PerAvg / FedProx / FedAMP / FedPHP
    parser.add_argument('-bt', "--beta", type=float, default=0.0,
                        help="Average moving parameter for pFedMe, Second learning rate of Per-FedAvg, \
                        or L1 regularization weight of FedTransfer")
    parser.add_argument('-lam', "--lamda", type=float, default=1.0,
                        help="Regularization weight for pFedMe and FedAMP")
    parser.add_argument('-mu', "--mu", type=float, default=0,
                        help="Proximal rate for FedProx")
    parser.add_argument('-K', "--K", type=int, default=5,
                        help="Number of personalized training steps for pFedMe")
    parser.add_argument('-lrp', "--p_learning_rate", type=float, default=0.01,
                        help="personalized learning rate to caculate theta aproximately using K steps")
    # FedFomo
    parser.add_argument('-M', "--M", type=int, default=5,
                        help="Server only sends M client models to one client at each round")
    # FedMTL
    parser.add_argument('-itk', "--itk", type=int, default=4000,
                        help="The iterations for solving quadratic subproblems")
    # FedAMP
    parser.add_argument('-alk', "--alphaK", type=float, default=1.0, 
                        help="lambda/sqrt(GLOABL-ITRATION) according to the paper")
    parser.add_argument('-sg', "--sigma", type=float, default=1.0)
    # APFL
    parser.add_argument('-al', "--alpha", type=float, default=1.0)
    # Ditto / FedRep
    parser.add_argument('-pls', "--plocal_steps", type=int, default=1)
    # MOON
    parser.add_argument('-ta', "--tau", type=float, default=1.0)
    # FedBABU
    parser.add_argument('-fts', "--fine_tuning_steps", type=int, default=1)
    # APPLE
    parser.add_argument('-dlr', "--dr_learning_rate", type=float, default=0.0)
    parser.add_argument('-L', "--L", type=float, default=1.0)
    #FedTrans
    parser.add_argument('-ere', "--every_recluster_eps", type=int, default=5)
    parser.add_argument('-ed', "--emb_dim", type=int, default=128)
    parser.add_argument('-alr', "--attn_learning_rate", type=float, default=0.005)
    parser.add_argument('-ncl', "--num_cluster", type=int, default=10)

    
    args = parser.parse_args(args=["-data","mnist", "-m", "cnn", -algo FedTrans -gr 2500 -did 0 -go cnn -nc 2")
    os.environ["CUDA_VISIBLE_DEVICES"] = args.device_id

    if args.device == "cuda" and not torch.cuda.is_available():
        print("\ncuda is not avaiable.\n")
        args.device = "cpu"

    print("=" * 50)

    print("Algorithm: {}".format(args.algorithm))
    print("Local batch size: {}".format(args.batch_size))
    print("Local steps: {}".format(args.local_steps))
    print("Local learing rate: {}".format(args.local_learning_rate))
    print("Total number of clients: {}".format(args.num_clients))
    print("Clients join in each round: {}".format(args.join_ratio))
    print("Client drop rate: {}".format(args.client_drop_rate))
    print("Time select: {}".format(args.time_select))
    print("Time threthold: {}".format(args.time_threthold))
    print("Global rounds: {}".format(args.global_rounds))
    print("Running times: {}".format(args.times))
    print("Dataset: {}".format(args.dataset))
    print("Local model: {}".format(args.model))
    print("Using device: {}".format(args.device))

    if args.device == "cuda":
        print("Cuda device id: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    print("=" * 50)


    # if args.dataset == "mnist" or args.dataset == "fmnist":
    #     generate_mnist('../dataset/mnist/', args.num_clients, 10, args.niid)
    # elif args.dataset == "Cifar10" or args.dataset == "Cifar100":
    #     generate_cifar10('../dataset/Cifar10/', args.num_clients, 10, args.niid)
    # else:
    #     generate_synthetic('../dataset/synthetic/', args.num_clients, 10, args.niid)

    # with torch.profiler.profile(
    #     activities=[
    #         torch.profiler.ProfilerActivity.CPU,
    #         torch.profiler.ProfilerActivity.CUDA],
    #     profile_memory=True, 
    #     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
    #     ) as prof:
    # with torch.autograd.profiler.profile(profile_memory=True) as prof:
    server = run(args)

    
    # print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))
    # print(f"\nTotal time cost: {round(time.time()-total_start, 2)}s.")


In [ ]:
import torch
res = torch.load("cel.pt")
emb_list, weights = res
print(emb_list, weights)

In [ ]:
print(weights.size())

In [ ]:

x = torch.cat(emb_list, dim=0).squeeze(1)
print(x.size())

In [ ]:
from flcore.servers.serverfedtrans import Attn_Model
device = "cuda:0"
x.to(device)
attn_model = Attn_Model().to(device)
weights = attn_model(x)

In [ ]:
print(weights)

In [ ]:
res = torch.load("res.pt")

In [ ]:
iter_e, iter_w = res['inter_clusters_res']

In [ ]:
print(iter_e[1].size())

x = torch.cat(iter_e, dim=0).squeeze(1)

In [ ]:
print(x)

In [ ]:
import torch
from torch import nn

class Attn_Model_C(nn.Module):
    def __init__(self, emb_dim=128, attn_dim=128, num_heads=8):
        super(Attn_Model_C, self).__init__()
        self.emb_dim = emb_dim
        self.attn_dim = attn_dim
        self.query = nn.Linear(emb_dim, attn_dim)
        self.key = nn.Linear(emb_dim, attn_dim)
        #self.inter_LN = nn.LayerNorm(attn_dim)

        # 1-layer attention for simple verify

    def forward(self, x, models=None, prev_models=None):
        #x = self.inter_LN(x) 
        q = self.query(x)

        k = self.key(x)
        print("q:{}\n{}\nk:{}".format(q,"-"*5,k))
        scores = torch.matmul(q, k.transpose(-2, -1)) 
        #scores = torch.matmul(q, k.transpose(-2, -1)) / (self.attn_dim ** 0.2)
        print(scores)
        attention_weights = torch.softmax(scores, dim=-1)
        return attention_weights

In [ ]:

attn_model_1 = Attn_Model_C().to(device)
w = attn_model(x.to(device))

print(w)

In [ ]:
for c in res['intra_clusters_res']:
    if c is not None:
        c_e = c[0]
    else:
        continue
    
    x = torch.cat(c_e, dim=0).squeeze(1)
    w = attn_model_1(x.to(device))
    print(w)

In [ ]:

print(res['intra_clusters_res'][1])

In [ ]:
import torch
model = torch.load("gm_avg.pt")

In [ ]:
import copy
ps = []
param = [p.view(-1) for p in model.parameters()]
ps = torch.concat(param, dim=0)

In [ ]:
ps.size()

In [ ]:
ps[0]

## 测试attn参数backward

In [1]:
import torch
from torch import nn

model1 = torch.nn.Linear(10, 20)
model2 = torch.nn.Linear(10, 20)
emb_layer = torch.nn.Linear(220, 128)
for p_1, p_2 in zip(model1.parameters(), model2.parameters()):
    p_1.data += p_2.data
loss = nn.MSELoss()

In [2]:
class Attn_Model(nn.Module):
    def __init__(self, emb_dim=128, attn_dim=128, num_heads=8):
        super(Attn_Model, self).__init__()
        self.emb_dim = emb_dim
        self.attn_dim = attn_dim
        self.query = nn.Linear(emb_dim, attn_dim)
        self.key = nn.Linear(emb_dim, attn_dim)
        #self.inter_LN = nn.LayerNorm(attn_dim)

        # 1-layer attention for simple verify

    def forward(self, x, models=None, prev_models=None):
        #x = self.inter_LN(x) 
        q = self.query(x)
        k = self.key(x)

        scores = torch.matmul(q, k.transpose(-2, -1)) 

        #scaled coef removed since we want to diff weight matrix entries
        #scores = torch.matmul(q, k.transpose(-2, -1)) / (self.attn_dim ** 0.5)
        attention_weights = torch.softmax(scores, dim=-1)
        return attention_weights

In [3]:

import copy

##
def w_add_params(global_model, m_weights, models_params, require_grad=False):
    params = [] 
    res = copy.deepcopy(global_model)
    for param in res.parameters():
        param.data.zero_()
            
    for w, model_params in zip(m_weights, models_params):
        for res_param, model_param in zip(res.parameters(), model_params):
            if require_grad:
                res_param.grad += model_param.grad.clone().detach() * w
            res_param.data += model_param.data.clone().detach() * w
    return res
        
        

def emb(phead,emb_layer):
    params = []
    for p in phead.parameters():
        params.append(p.flatten())
    params = torch.cat(params)
    return emb_layer(params)


def weight_flatten(model):
    params = []
    for u in model.parameters():
        params.append(u.view(-1))
    params = torch.cat(params)

    return params

### 创建模型以及计算weighted sum

In [4]:
models = [nn.Linear(10,20) for i in range(10)]
flatten_models = [weight_flatten(model) for model in models]
emb_layer = nn.Linear(len(flatten_models[0]), 128)
models_emb = [emb(model, emb_layer) for model in models]


In [5]:
x = torch.cat([e.reshape(1,-1) for e in models_emb], dim=0)
print(x.size())

torch.Size([10, 128])


In [6]:
attn_model = Attn_Model()

weights = attn_model(x)
print(weights)

tensor([[0.1003, 0.0944, 0.0972, 0.0976, 0.1033, 0.0949, 0.0961, 0.1086, 0.1007,
         0.1069],
        [0.1040, 0.0960, 0.1009, 0.1006, 0.1091, 0.0934, 0.0980, 0.1002, 0.0944,
         0.1034],
        [0.1015, 0.1016, 0.0952, 0.0979, 0.1074, 0.0894, 0.0946, 0.1017, 0.1046,
         0.1061],
        [0.1006, 0.0991, 0.0998, 0.1031, 0.1122, 0.0862, 0.0878, 0.1057, 0.1042,
         0.1011],
        [0.1013, 0.1000, 0.0974, 0.1025, 0.1032, 0.0967, 0.1020, 0.1014, 0.0945,
         0.1011],
        [0.0960, 0.0962, 0.1056, 0.0968, 0.1139, 0.0991, 0.0951, 0.1032, 0.1033,
         0.0909],
        [0.1000, 0.0968, 0.1044, 0.0969, 0.1099, 0.0955, 0.0956, 0.1053, 0.0997,
         0.0960],
        [0.1124, 0.1011, 0.0952, 0.1032, 0.1070, 0.0931, 0.0856, 0.0987, 0.0987,
         0.1051],
        [0.1021, 0.0969, 0.0977, 0.0948, 0.1059, 0.0897, 0.0959, 0.1105, 0.1027,
         0.1038],
        [0.1034, 0.0979, 0.0991, 0.0984, 0.1088, 0.0927, 0.0896, 0.1032, 0.0980,
         0.1090]], grad_fn=<

In [ ]:
new_ms = []
for i in range(weights.size()[0]):
    w = [weights[i][j] for j in range(weights[i].size()[0])]
    new_m = w_add_params(models[0], w, [model.parameters() for model in models])
    new_ms.append(new_m)
print(len(new_ms))

In [ ]:
vecs = nn.utils.parameters_to_vector(models[0].parameters())


### 更新模型

In [ ]:
from collections import defaultdict, OrderedDict

In [ ]:

models = new_ms
inner_optimizers = [torch.optim.SGD(models[i].parameters(),\
                                lr=0.005) for i in range(10)]
inner_states = [m.state_dict() for m in models]

for i in range(10):
    #one-step local training
    inp = torch.rand([10])
    y = torch.rand([20])
    mseloss = torch.nn.MSELoss()
    #print("prev_model:{}".format(nn.utils.parameters_to_vector(models[i].parameters())))
    output = models[i](inp)
    loss = mseloss(output, y)
    loss.backward()
    inner_optimizers[i].step()
    #print("cur_model:{}".format(nn.utils.parameters_to_vector(models[0].parameters())))
final_states = [m.state_dict() for m in models]
delta_thetas = [OrderedDict({k: inner_states[i][k] - final_states[i][k] for k in models[i].state_dict().keys()}) for i in range(10)]

#### 测试模型参数修改函数

In [ ]:
from collections import OrderedDict
model = models[0]
# 我想修改的模型state
sd = model.state_dict()
w = weights[0]
# 用一个带grad的weight去aggregate模型参数
# 把参数放回state，保存带梯度的参数到另外一个list，用于梯度计算
print(sd)

def w_add_parameters(sd, w, models):
    sg = OrderedDict()
    for w_i, model in zip(w, models):
        for key in sd.keys():
            if key not in sg.keys():
                sg[key] = w_i * model.state_dict()[key]
            else:
                sg[key] += w_i * model.state_dict()[key]
            print(sg)
            sd[key] += sg[key].data
    return sg, sd
sg, sd = w_add_parameters(sd, w, models)



#### 更新模型

In [20]:

model = models[0]
# 我想修改的模型state
sd = model.state_dict()
w = weights[0]
# 用一个带grad的weight去aggregate模型参数
# 把参数放回state，保存带梯度的参数到另外一个list，用于梯度计算

sg, sd = w_add_parameters(sd, w, models)

inner_state = sd
nm = nn.Linear(10,20)
nm.load_state_dict(sd)
inner_optimizer = torch.optim.SGD(nm.parameters(),lr=0.05)
inner_state = nm.state_dict()
for _ in range(10):
    inp = torch.rand([10])
    y = torch.rand([20])
    mseloss = torch.nn.MSELoss()
    #print("prev_model:{}".format(nn.utils.parameters_to_vector(models[i].parameters())))
    output = nm(inp)
    loss = mseloss(output, y)
    loss.backward()
    inner_optimizer.step()
final_state = nm.state_dict()
delta_theta = OrderedDict({k:inner_state[k]-final_state[k] for k in nm.state_dict().keys()})

lv = list(sg.values())
param_list = list(attn_model.parameters())
param_list.extend(emb_layer.parameters())
torch.autograd.grad(lv,param_list,grad_outputs=list(delta_theta.values()))

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

### 计算JVP


In [ ]:
lr = 0.01
attn_optimizer = torch.optim.SGD(
    [
        {'params' : [p for p in attn_model.parameters()]} ,
        {'params' : [p for p in emb_layer.parameters()]}
    ], lr=lr, momentum=0.9)

param_list = list(attn_model.parameters())
param_list.extend(emb_layer.parameters())
attn_optimizer.zero_grad()

In [ ]:
grads = [
            torch.autograd.grad([p for p in model.parameters()], param_list , grad_outputs=list(delta_theta.values()))\
            for model, delta_theta in zip(models,delta_thetas)
        ]      
for grad in grads:
    for param, g in zip(param_list, grad):
        param.grad += grad 

torch.nn.utils.clip_grad_norm_(param_list, 50)
attn_optimizer.step()

In [ ]:
inner_states = model[0]
final_states = .state_dict()

# calculating delta theta
delta_theta = OrderedDict({k: inner_state[k] - final_state[k] for k in weights.keys()})

# calculating phi gradient
hnet_grads = torch.autograd.grad(
    list(weights.values()), hnet.parameters(), grad_outputs=list(delta_theta.values())
)

print(torch.cat(g_1,dim=0).reshape(-1,1))

In [ ]:
print(models[0].state_dict().values())

In [ ]:
mo = nn.Linear(10,20)
mo.load_state_dict(new_ms[0].state_dict())
print(mo)

In [ ]:
for p in mo.parameters():
    print(p)

In [ ]:
import random
import copy
res = copy.deepcopy(model1)
for param in res.parameters():
    param.data.zero_()

for rp, p in zip(res.parameters(), model1.parameters()):
    rp.data += p.data.clone()
    

print(res)

In [ ]:

res.clone().detach().requires_grad_(True)